# ML Module documentation

### About
This IPython notebook uses quantum-chemistry outputs of third-party programs to train a Kernel Ridge Regression (KRR) Machine Learning Model. The goal of this Model is to use low-accuracy data (e.g. semi-empirical calculations) to predict higher-accuracy data (e.g. excitation energies in TDDFT quality). The notebook requires that the underlying database has been set up consistently for use with the ArchOnML package. This can be achieved by generating it with the related 01...ipynb notebook. Note, that the ML Module will work irrespective of whether this is a conformational space or chemical space scan.

The notebook is divided in two different sections that can run independent from each other. The sections and their purposes are:

# 
### Part A - Training and Saving a ML Model.
Part A starts with defining what how data should be prepared for machine learning. This is followed by loading the precomputed Training data and forming descriptors out of the raw data. Note, that at this point also the labels should be available for the training set (i.e. the property you want to learn to predict). The file containing the label data should be placed inside the SemiEmpData folder with the same name as the Label. For more details, please refer to the documentation.
After that, the distance matrices between all entries are precalculated and stored by feeding the entries into a DataTens object. This object is capable of splitting the data into training/testing sets of user-defined sizes and to prepare the data in K stratified blocks for K-fold cross-validation of hyperparameters.
Finally, the MLModel object is created and trained in a (here 5-fold) cross-validation. Here, each hyperparameter scan is a separate program block to monitor each of the somewhat time-consuming steps carefully. Finally, the model is finalized and stored to hard disk.

# 
### Part B - Loading and Predicting with a ML Model.
Part B is an example for how to load a previously trained model and run predictions. Note that this part can be run fully independent from Part A, , allowing to run predictions in parallel batches on a high-performance computing cluster.

In [ ]:
import os, sys, time
from copy import deepcopy as dc

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
from rdkit.Chem.Draw import IPythonConsole

from archonml.entries import MLEntry
from archonml.desctools import Descriptor
from archonml.krr_tens import DataTens
from archonml.krr_model import MLModel
from archonml.utils import timeconv

# Console Options / Quality of Life improvements
plt.rcParams['figure.figsize'] = [4, 3]
plt.rcParams['figure.dpi'] = 200 # 200 e.g. is really fine, but slower
IPythonConsole.ipython_useSVG=True
IPythonConsole.drawOptions.addAtomIndices = True
IPythonConsole.molSize = 400, 400
np.set_printoptions(suppress=True)

# PART A Cells

# Step 1 - Inputs for setting up the ML entries and descriptors.

In [ ]:
# Load the folder hierarchy information of the current project from DBSpecs.cfg (generated by 01_DB_Starter.ipynb)
MLEntry.get_db_specs()

# Specify, what the data to train on looks like.
# Note that the MOWin attribute is somewhat special, as it not only defines how many orbitals around the HOMO-LUMO orbitals are to be considered, but it also determines how many will be written to the
# native MergeFile when wrtiting them for the first time.
LType         = "Training"
LQC_Pack      = "orca"
LQC_low       = "PM3"
LQC_high      = "TD-DFT"
LLabelL       = [""]
LDataL        = ["Geometry", "Mulliken_F", "SEmpOrbInfo_F"]
MLEntry.MOWin = 4

# Predefine which Descriptors should be used and set up the Descriptor Instance.
# Note that each descriptor will require at least N**2 distances to be calculated, with N being the number of training entries.
# Further, the amount of training data scales with approximately 2*(N**2)*(MOWin**2) if orbital data is to be used.
LDescL    = ["SEmpOrbEnDiffs", "SEmpOccs", "SEmpVirs", "SEmpTups", "SEmpEigCoul", "SEmpOccEigCoul",
             "SEmpNEl", "SEmpOccPCMEigCoul", "SEmpVirPCMEigCoul", "SEmpTransPCMEigCoul", "SEmpOccVirPTransSum", "SEmpHOLUPDiff"]
DescInst  = Descriptor(LDescL)
MLOBJLIST = []

In [ ]:
# Read the Training IDs and Fingerprints from a Calculation Library. Initialize the MLEntry objects.
# If you have several calculation libraries, just copy this cell and run it with another file opened as FID.
FID = open("./SampleCalcs_1_PROJ", "r")
FLoc = FID.readlines()
FID.close()
LID      = []
LFingerP = []
for line in range(len(FLoc)):
    LID.append(FLoc[line].split()[0])
    Aux     = FLoc[line].split()[1]
    FingLen = len(Aux.split(","))
    LFing = []
    for jj in range(FingLen):
        LFing.append(int(Aux.split(",")[jj]))
    LFingerP.append(LFing)
    LObj = MLEntry(LID[-1], LType, LFingerP[-1], LQC_Pack, LQC_low, LQC_high, LDataL, LLabelL)
    MLOBJLIST.append(dc(LObj))

In [ ]:
# Only needed once per "newly calculated entry": Merge raw data into the native format of ArchOnML.
cnt   = 0
tTot  = 0
incnt = 0
mReq  = 0
now   = time.time()
for ii in range(len(MLOBJLIST)):
    MLOBJLIST[ii].merge_data()
    cnt  += 1
    Perc  = cnt / len(MLOBJLIST)
    UpdThrsh = 0.001
    if Perc > UpdThrsh+(incnt*UpdThrsh):
        then  = time.time()
        tReq  = then-now
        tTot += tReq
        mReq  = tTot / cnt
        Rem   = float(len(MLOBJLIST)-cnt)*mReq
        clear_output(wait=True)
        STR1  = "Finished {:.2f} % ({}) of all structures ({}) in {:.1f} seconds. ({})\n".format(Perc*100, cnt, len(MLOBJLIST), tTot, timeconv(tTot),)
        STR2  = "Required {:.3f} seconds on average for each structure.\n".format(mReq)
        STR3  = "Expecting {:.1f} seconds remaining. ({})\n".format(Rem, timeconv(Rem))
        print(STR1+STR2+STR3)
        incnt += 1
        now    = time.time()

# Step 2 - Initialise Descriptors for every MLEntry

In [ ]:
# Get the MLEntry.Data from MergeFiles and Describe immediately.
# This step can be commented out, if a "described MLOBJLIST" has been saved to a pickle file already (see below). 
cnt   = 0
tTot  = 0
incnt = 0
now   = time.time()
for ii in range(len(MLOBJLIST)):
    MLOBJLIST[ii].get_merged_data()
    DescInst.describe(MLOBJLIST[ii])
    CurID = MLOBJLIST[ii].ID
    print(CurID)
    cnt  += 1
    Perc  = cnt / len(MLOBJLIST)
    UpdThrsh = 0.001
    if Perc > UpdThrsh+(incnt*UpdThrsh):
        then  = time.time()
        tReq  = then-now
        tTot += tReq
        mReq  = tTot / cnt
        Rem   = float(len(MLOBJLIST)-cnt)*mReq
        clear_output(wait=True)
        STR1  = "Finished {:.2f} % ({}) of all structures ({}) in {:.1f} seconds. ({})\n".format(Perc*100, cnt, len(MLOBJLIST), tTot, timeconv(tTot),)
        STR2  = "Required {:.3f} seconds on average for each structure.\n".format(mReq)
        STR3  = "Expecting {:.1f} seconds remaining. ({})\n".format(Rem, timeconv(Rem))
        print(STR1+STR2+STR3)
        incnt += 1
        now    = time.time()

In [ ]:
# Save a "described MLOBJLIST" to a pickle file. Only needed when first saving the file, or if the set changes.
# Can be commented out afterwards.

import pickle
PICKLE = "LearnSet5k.pkl"
with open(PICKLE, "wb") as OID:
    for ii in range(len(MLOBJLIST)):
        pickle.dump(MLOBJLIST[ii], OID)

In [ ]:
# # Load a "described MLOBJLIST" from a pickle file. Should be uncommented, if a file exists.

# import pickle
# PICKLE = "LearnSet5k.pkl"
# AUX = []
# with open(PICKLE, "rb") as FID:
#     for ii in range(len(MLOBJLIST)):
#         AUX.append(pickle.load(FID))
# MLOBJLIST = dc(AUX)

# Step 3 - Turning Descriptors into KRR tensor format via DataTens instance.
Includes Setting up the stratified Cross-Validation IDs.

In [ ]:
# Initialize the DataTens Object. Below are the default options for data management.
DataTens.CVGridPts    = 64
DataTens.CVSize       = 5
DataTens.RandomStrat  = True
DataTens.FixedSeed    = True
DataTens.MinMod       =  0.01   # These have been changed for the conformer scan.
DataTens.MaxMod       =  1000   # These have been changed for the conformer scan.
DataTens.Lambda_Bot   =  -10
DataTens.Lambda_Top   =  -1
DataTens.BinWinThrsh  = 10
DataTens.BinWinEscape = 100

# At initialization, it will precalculate all distance matrices.
DataInst = DataTens(MLOBJLIST, mem_mode="high")

In [ ]:
# Split the data into Training and Test set.
# Given Percentage is the percentage of the training set.
# Whether an entry is considered Test or Training is written to DataInst.Status
DataInst.train_test_split(0.8)

In [ ]:
# Set a random subportion Active / Inactive. By changing this percentage you obtain the different points on a learning curve.
DataInst.train_test_split(_, UPDATE=True, ActPerc=1.0)

In [ ]:
# K-Fold Stratify the Data with respect to the desired Label.
# This leads to the formation of the CVIDBlocks.
DataInst.stratify("")

In [ ]:
# Visualize the stratified entries.
# Checking the distributions:
LabelBlocks = []
for ii in range(DataTens.CVSize):
    LocList = DataInst.CVIDBlocks[ii]
    Aux = []
    for jj in range(len(LocList)):
        Aux.append(DataInst.Labels[""][LocList[jj]])
    LabelBlocks.append(Aux)
counts, bins, bars = plt.hist(LabelBlocks, bins=15) 

# Step 4 - Cross-Validation Training for a ML-Model w.r.t. a desired Label.

In [ ]:
# Initialize a Model with respect to one of the Labels.
KRR = MLModel("")

In [ ]:
# Build the CV Tensor for training against Hyperparameters.
# Here, Block "0" for validation.
CV_VaIDs = DataInst.CVIDBlocks[0]
CV_TrIDs = DataInst.CVIDBlocks[1] + DataInst.CVIDBlocks[2] + DataInst.CVIDBlocks[3] + DataInst.CVIDBlocks[4]
DataInst.update_cv_tens(CV_TrIDs, CV_VaIDs, MLOBJLIST)

# Determine the best Hyperparameters SIGMA and LAMBDA for this CV step
KRR.cv_para(DataInst, MLOBJLIST, optimizer="vectorized", convR=0.001, convM=0.00001, MaxIter=4, ignoreConv=True)

In [ ]:
# Build the CV Tensor for training against Hyperparameters.
# Here, Block "1" for validation.
CV_VaIDs = DataInst.CVIDBlocks[1]
CV_TrIDs = DataInst.CVIDBlocks[0] + DataInst.CVIDBlocks[2] + DataInst.CVIDBlocks[3] + DataInst.CVIDBlocks[4]
DataInst.update_cv_tens(CV_TrIDs, CV_VaIDs, MLOBJLIST)

# Determine the best Hyperparameters SIGMA and LAMBDA for this CV step
KRR.cv_para(DataInst, MLOBJLIST, optimizer="vectorized", convR=0.001, convM=0.00001, MaxIter=4, ignoreConv=True)

In [ ]:
# Build the CV Tensor for training against Hyperparameters.
# Here, Block "2" for validation.
CV_VaIDs = DataInst.CVIDBlocks[2]
CV_TrIDs = DataInst.CVIDBlocks[0] + DataInst.CVIDBlocks[1] + DataInst.CVIDBlocks[3] + DataInst.CVIDBlocks[4]
DataInst.update_cv_tens(CV_TrIDs, CV_VaIDs, MLOBJLIST)

# Determine the best Hyperparameters SIGMA and LAMBDA for this CV step
KRR.cv_para(DataInst, MLOBJLIST, optimizer="vectorized", convR=0.001, convM=0.00001, MaxIter=4, ignoreConv=True)

In [ ]:
# Build the CV Tensor for training against Hyperparameters.
# Here, Block "3" for validation.
CV_VaIDs = DataInst.CVIDBlocks[3]
CV_TrIDs = DataInst.CVIDBlocks[0] + DataInst.CVIDBlocks[1] + DataInst.CVIDBlocks[2] + DataInst.CVIDBlocks[4]
DataInst.update_cv_tens(CV_TrIDs, CV_VaIDs, MLOBJLIST)

# Determine the best Hyperparameters SIGMA and LAMBDA for this CV step
KRR.cv_para(DataInst, MLOBJLIST, optimizer="vectorized", convR=0.001, convM=0.00001, MaxIter=4, ignoreConv=True)

In [ ]:
# Build the CV Tensor for training against Hyperparameters.
# Here, Block "4" for validation.
CV_VaIDs = DataInst.CVIDBlocks[4]
CV_TrIDs = DataInst.CVIDBlocks[0] + DataInst.CVIDBlocks[1] + DataInst.CVIDBlocks[2] + DataInst.CVIDBlocks[3]
DataInst.update_cv_tens(CV_TrIDs, CV_VaIDs, MLOBJLIST)

# Determine the best Hyperparameters SIGMA and LAMBDA for this CV step
KRR.cv_para(DataInst, MLOBJLIST, optimizer="vectorized", convR=0.001, convM=0.00001, MaxIter=4, ignoreConv=True)

In [ ]:
# Finalize the Model - take the mean best SIGMA and LAMBDA across all CV steps and train against the full CV set.
# Finally try to predict the values of the Test set - and compare how well the trained model does.
KRR.final_cv(DataInst, MLOBJLIST)

In [ ]:
# Store an overview of the final performance.
OID = open("FinStats.out", "w") 
OID.write("# Num        Label            PredVal_M        PredVal_R\n")
for ii in range(len(KRR.FinTestLabelVals)):
    OID.write("{:5d}       {: 3.7f}       {: 3.7f}       {: 3.7f}\n".format(ii, KRR.FinTestLabelVals[ii], KRR.FinPredMVals[ii], KRR.FinPredRVals[ii]))
OID.close()

# Step 5 - Saving the ML-Model.

In [ ]:
KRR.save_model("")

# PART B
This part can be used to interactively run a test prediction. Note that for actual production, the batchable script "03" should be used.

# Step 1 - Loading an ML-Model.

In [ ]:
# Load an existing Model and setup the MLOBJLIST as wells as the Descriptor instance.
# Note that the MLOBJLIST contains only those Training entries at this point, that were actually used in this exact succession during training the original model.
# (Also note, that the succession is in fact in rising order of entries, as the final training will sort them when going through range(0 ... NActive).)
MLEntry.get_db_specs()
LLabelL   = [""]
KRR = MLModel(LLabelL)
MLOBJLIST, DescInst = KRR.get_model("")

In [ ]:
# Load the Data to Predict.
LType     = "Prediction"
LQC_Pack  = "orca"
LQC_low   = "PM3"
LQC_high  = "TD-DFT"
LDataL    = ["Geometry", "Mulliken_F", "SEmpOrbInfo_F"]
# Read the Prediction IDs and Fingerprints from File. Append to the pre-loaded MLEntries.
FID = open ("", "r")               # add filename here
FLoc = FID.readlines()
FID.close()
LID      = []
LFingerP = []
PrepList = []
for line in range(len(FLoc)):
    LID.append(FLoc[line].split()[0])
    Aux     = FLoc[line].split()[1]
    FingLen = len(Aux.split(","))
    LFing = []
    for jj in range(FingLen):
        LFing.append(int(Aux.split(",")[jj]))
    LFingerP.append(LFing)
    LObj = MLEntry(LID[-1], LType, LFingerP[-1], LQC_Pack, LQC_low, LQC_high, LDataL, LLabelL)
    PrepList.append(dc(LObj))
# Only needed once per "newly calculated entry": Merge Data into my native format
for ii in range(len(PrepList)):
    PrepList[ii].merge_data()

In [ ]:
# Get the MLEntry.Data from MergeFiles and immediately describe.
cnt   = 0
tTot  = 0
incnt = 0
now   = time.time()
for ii in range(len(MLOBJLIST)):
    MLOBJLIST[ii].get_merged_data()
    DescInst.describe(MLOBJLIST[ii])
    CurID = MLOBJLIST[ii].ID
    print(CurID)
    cnt  += 1
    Perc  = cnt / len(MLOBJLIST)
    UpdThrsh = 0.001
    if Perc > UpdThrsh+(incnt*UpdThrsh):
        then  = time.time()
        tReq  = then-now
        tTot += tReq
        mReq  = tTot / cnt
        Rem   = float(len(MLOBJLIST)-cnt)*mReq
        clear_output(wait=True)
        STR1  = "Finished {:.2f} % ({}) of all structures ({}) in {:.1f} seconds. ({})\n".format(Perc*100, cnt, len(MLOBJLIST), tTot, timeconv(tTot),)
        STR2  = "Required {:.3f} seconds on average for each structure.\n".format(mReq)
        STR3  = "Expecting {:.1f} seconds remaining. ({})\n".format(Rem, timeconv(Rem))
        print(STR1+STR2+STR3)
        incnt += 1
        now    = time.time()

In [ ]:
# Instantiate the DataTens Object to get the precalculated Distance matrices. For predictions, no splitting or stratifying needed, of course.
DataInst = DataTens(MLOBJLIST)

In [ ]:
# Perform predictions from loaded model.
KRR.predict_from_loaded(DataInst, MLOBJLIST)

In [ ]:
# Save Predictions to file.
FID = open("{}.Predictions".format(LLabelL[0]), "w")
cnt = 0
for ii in range(len(MLOBJLIST)):
    if MLOBJLIST[ii].Type == "Prediction":
        Aux = ""
        for jj in range(len(MLOBJLIST[ii].FingerP)):
            Aux  += "{},".format(MLOBJLIST[ii].FingerP[jj])
            LFing = Aux.rstrip(Aux[-1])
        SSS = "{}   {}   {: 12.9f}\n".format(MLOBJLIST[ii].ID, LFing, KRR.Predictions[cnt])
        FID.write(SSS)
        cnt += 1
FID.close()